# Illustration of Alidaee, Auerbach, and Leung (2020)

This notebook walks through a brief illustration of the Python implementation of the penalized regression estimator of Alidaee, Auerbach, and Leung (2020) for recovering networks from ARD. Some interactive tutorials for Python, numpy, and the networkx module can be found [here](https://github.com/mpleung/Python_tutorials).

The code blocks in this notebook can be changed if this notebook is opened in [nteract](https://nteract.io/desktop) or the following binder link:

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/mpleung/ARD/master?filepath=Python%2Fwalkthrough.ipynb) 

Either method allows you to execute any Python code block by clicking on it and hitting *Shift+Enter*. These blocks can be modified by the user.

## Background

Let N2 be the number of agents in the full network, N1 the number of sample agents from which the researcher will gather ARD, and K the number of ARD questions of the form "How many of your friends have type k?" for k = 1, ..., K. Types could be characteristics like gender, race, or other such traits. The goal is to recover the N2 x N1 matrix $M^*$, which gives the linking probability between agents in the N1 sample and agents in the N2 population. 

The researcher conducts a full census of the network to obtain type data, which, in the example below, is stored in type\_data.csv, a N2 x K matrix of indicators, where the ik-th entry is an indicator for whether agent i is type k. The ARD data is stored in ARD\_data.csv, an N1 x K matrix where the ik-th entry is the number of agent i's friends of type k.

## Setup

First we'll import the required modules.

In [1]:
import numpy as np, pandas as pd, nuclear_norm_module as nn
np.random.seed(0) # set seed

*numpy* is a module for working with matrices in Python. 

*pandas* is a module only used here to load CSVs. 

*nuclear_norm_module* is our implementation of our estimator.

### Simulate Data

This section of the tutorial can be skipped. It is included only to show how the CSVs to be loaded below are generated.

Let *N1* be the number of units for which we obtain ARD. Let *N2* be the population size, meaning the number of units in the entire network. Let *K* be the number of traits for which we collect ARD.

We simulate an undirected network with no self links from a random dot product graph on *N2* units. Then we take the submatrix of the network corresponding to links involving the subset of *N1* units for which we want to generate ARDs.

In [3]:
N1 = 100
N2 = 200
K = int(round(N1**(0.4)))

# simulate network 
positions = np.sqrt(np.random.uniform(0,1,N2))
M = positions * positions[:,None] # n x n matrix of link probabilities
np.fill_diagonal(M,0) # zero out diagonal entries to ensure no self links
U = np.random.uniform(0,1,size=(N2,N2))
U = U.T/2 + U/2 # make matrix symmetric to have an undirected network
G = (U < M)[:,:N1] # simulated network submatrix

When we discuss below how to use the result of our estimation procedure, we explain some of these commands used to simulate the network.

Next we create a *K* by *N2* matrix of types, which we'll just take to be a matrix of i.i.d. Bernoulli random variables. Call this matrix *types*. Then we'll construct the ARDs, which is what the econometrician actually observes in data. This is obtained just by taking the matrix product of *types* and *G*.

In [4]:
types = np.random.binomial(1,0.5,size=(K,N2))
ARDs = types.dot(G)

In preparation for the actual exercise, we next save these matrices in CSV files.

In [5]:
pd.DataFrame(ARDs.T).to_csv('ARD_data.csv', index=False, header=False)
pd.DataFrame(types.T).to_csv('type_data.csv', index=False, header=False)

This saves *ARDs* in a file *ARD_data.csv* and *types* in a file *type_data.csv*.

### Load Data

In the previous section, we simulated the following numpy matrices, which we saved as CSVs.

*types* is a K by N2 matrix of indicators giving the types of each unit in the surveyed population. For example, the first column might contain indicators for being female, being white, etc. for unit 1.

*ARDs* is a K by N1 matrix of K ARDs for N1 observations. For example, the first column might contain the number of female friends, number of white friends, etc. of unit 1.

In practice, we would have gathered this data from the field and have them already saved in CSVs. We load them as follows.

In [2]:
# load CSVs as numpy matrices
ARDs = pd.read_csv('ARD_data.csv', header=None).values.T
types = pd.read_csv('type_data.csv', header=None).values.T

# store dimensions
K = ARDs.shape[0]
N1 = ARDs.shape[1]
N2 = types.shape[1]

## Estimation

To estimate the distribution of the network, we use the function *matrix_regression* in *nuclear_norm_module*, which was imported above under the abbreviation *nn*.

Note: the code snippet below might take a beat to run in binder, but it is fast on a personal computer.

In [3]:
M_hat = nn.matrix_regression(ARDs, types)

*M_hat* is our estimate, an N2 by N1 matrix, where the *ij*th entry is our estimate of the probability that unit *i* links with unit *j*.

Just to see that we've done something, let's print the upper 5 by 5 submatrix of *M_hat*.

In [4]:
print(M_hat[0:5,0:5])

[[0.         0.63775324 0.44260486 0.53027242 0.44479568]
 [0.63775324 0.         0.58602635 0.68282686 0.56840426]
 [0.44260486 0.58602635 0.         0.4747138  0.40486628]
 [0.53027242 0.68282686 0.4747138  0.         0.47632266]
 [0.44479568 0.56840426 0.40486628 0.47632266 0.        ]]


## Using the Result

The estimated network distribution *M_hat* can be saved in a CSV as follows. It can then be imported into the user's favorite statistical computing environment.

In [9]:
# save in a CSV file called estimated_network
pd.DataFrame(M_hat).to_csv('estimated_network.csv', index=False, header=False)

This result can then be used as an input into a second-stage model. See Breza et al. (2019) for possible applications.

It may also be of interest to simulate networks from *M_hat*. In our model, *M_hat* (or more precisely its upper diagonal, given the network is undirected) is a matrix of independent linking probabilities. Thus to simulate a network *G* from *M_hat*, we just draw an N2 by N1 matrix of i.i.d. uniform random variables, denoted by *U* and form the *ij* entry of *G* according to $G_{ij}=\mathbf{1}\{U_{ij} < \hat{M}_{ij}\}$. In Python, this is done as follows.

First, draw an N2 by N1 matrix of i.i.d. uniform random variables.

In [5]:
U = np.random.uniform(0,1,size=(N2,N1)) # draw uniform random variables
np.fill_diagonal(U,0)                   # zero out the diagonal entries 
                                        #    (if no self links)

If the true network is undirected, then we need to symmetrize the upper N1 by N1 submatrix. If the true matrix is directed, then skip this step.

In [6]:
U_sub = U[:N1,:]            # extract upper N1 x N1 submatrix
U_sub = U_sub.T/2 + U_sub/2 # symmetrize the submatrix
U[:N1,:] = U_sub            # replace the upper N1 x N1 submatrix of
                            #     the original matrix U with U_sub

Given *U* and *M_hat*, our simulated network *G* is obtained as follows.

In [7]:
G = (U < M_hat).astype('int')

The command *(U < M_hat)* generates an N2 x N1 matrix of booleans (True/False) where the *ij*th entry is True if and only if $U_{ij} < \hat{M}_{ij}$. To convert this to 1s and 0s, we use the *astype* method to convert to the integer type.

Now we can use *G* as an input into some second stage procedure. See Breza et al. (2019) for examples. Just to see that we've done something, let's print the upper 10 x 10 submatrix of *G*:

In [8]:
print(G[0:5,0:5])

[[0 1 0 0 0]
 [1 0 1 1 1]
 [0 1 0 0 0]
 [0 1 0 0 1]
 [0 1 0 1 0]]
